## Severe Weather Data Cleanup
#### CSVs come from: https://www.ncdc.noaa.gov/data-access

* 210 files total | 3 types
* Storm Event Details
* Storm Event Locations
* Storm Event Fatalities
* Dates: 1950-2019(Jan)

In [1]:
# Import Dependencies
import glob
import os
import pandas as pd

# Merging & Cleaning | StormEventsLocations CSVs

In [2]:
# Reading StormEventsLocations CSVs in using glob

path = '../Resources/StormData/StormEventsLocations'

lfiles = glob.glob(os.path.join(path, '*.csv'))

ldata = []
for lfile in lfiles:
    record = pd.read_csv(lfile)
    ldata.append(record)

In [3]:
# Concat CSVs into single DataFrame & check for duplicate columns
lfulldata = pd.concat(ldata, ignore_index=True)
lfulldata.head()

,YEARMONTH,EPISODE_ID,EVENT_ID,LOCATION_INDEX,RANGE,AZIMUTH,LOCATION,LATITUDE,LONGITUDE,LAT2,LON2
0,199702,2059155,5590910,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,199702,2059152,5590895,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,199702,2059152,5590896,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,199702,2059152,5590897,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,199702,2059152,5590898,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Drop unnecessary columns
lfulldata = lfulldata.drop(['RANGE', 'AZIMUTH'], axis=1)

In [5]:
# Rename columns
lfulldata = lfulldata.rename(columns={"LAT2":"LATITUDE2", "LON2":"LONGITUDE2"})

In [6]:
# Drop nulls in LOCATION column
lfulldata = lfulldata.dropna(axis=0, subset=['LOCATION'])

In [7]:
# Drop nulls in LATITUDE column
lfulldata = lfulldata.dropna(axis=0, subset=['LATITUDE'])

In [8]:
# Drop nulls in LATITUDE2 column
lfulldata = lfulldata.dropna(axis=0, subset=['LATITUDE2'])
lfulldata.head()

,YEARMONTH,EPISODE_ID,EVENT_ID,LOCATION_INDEX,LOCATION,LATITUDE,LONGITUDE,LATITUDE2,LONGITUDE2
12,199701,2058233,5591463,1,DOUGLAS,34.17,-86.38,3410.0,8623.0
20,199705,2057340,5585340,1,LAREDO,40.03,-93.45,4002.0,9327.0
21,199705,2057340,5585341,2,HUMPHREYS,40.08,-93.35,4005.0,9321.0
34,199705,2057340,5585342,1,PURDIN,40.10,-93.38,4006.0,9323.0
35,199705,2057340,5585343,1,WINIGAN,40.05,-92.88,4003.0,9253.0


In [9]:
# Splitting the YEARMONTH column to get separate YEAR & MONTH columns
lfulldata['YEARMONTH1'] = lfulldata['YEARMONTH'].astype(str)
lfulldata['YEAR'] = lfulldata['YEARMONTH1'].str[0:4]
lfulldata['MONTH'] = lfulldata['YEARMONTH1'].str[4:6]
lfulldata['YEAR'] = pd.to_numeric(lfulldata.YEAR, errors='coerce')
lfulldata['MONTH'] = pd.to_numeric(lfulldata.MONTH, errors='coerce')

lfulldata.head()

,YEARMONTH,EPISODE_ID,EVENT_ID,LOCATION_INDEX,LOCATION,LATITUDE,LONGITUDE,LATITUDE2,LONGITUDE2,YEARMONTH1,YEAR,MONTH
12,199701,2058233,5591463,1,DOUGLAS,34.17,-86.38,3410.0,8623.0,199701,1997,1
20,199705,2057340,5585340,1,LAREDO,40.03,-93.45,4002.0,9327.0,199705,1997,5
21,199705,2057340,5585341,2,HUMPHREYS,40.08,-93.35,4005.0,9321.0,199705,1997,5
34,199705,2057340,5585342,1,PURDIN,40.10,-93.38,4006.0,9323.0,199705,1997,5
35,199705,2057340,5585343,1,WINIGAN,40.05,-92.88,4003.0,9253.0,199705,1997,5


In [10]:
# Create a new column for DECADE using binning on the YEAR column
bins = [1950, 1960, 1970, 1980, 1990, 2000, 2010, 2019]
decades = ['1950-1959', '1960-1969', '1970-1979', '1980-1989', '1990-1999', '2000-2010', '2010-2019']

lfulldata['DECADE'] = pd.cut(lfulldata['YEAR'], bins, labels=decades)
lfulldata.head()

,YEARMONTH,EPISODE_ID,EVENT_ID,LOCATION_INDEX,LOCATION,LATITUDE,LONGITUDE,LATITUDE2,LONGITUDE2,YEARMONTH1,YEAR,MONTH,DECADE
12,199701,2058233,5591463,1,DOUGLAS,34.17,-86.38,3410.0,8623.0,199701,1997,1,1990-1999
20,199705,2057340,5585340,1,LAREDO,40.03,-93.45,4002.0,9327.0,199705,1997,5,1990-1999
21,199705,2057340,5585341,2,HUMPHREYS,40.08,-93.35,4005.0,9321.0,199705,1997,5,1990-1999
34,199705,2057340,5585342,1,PURDIN,40.10,-93.38,4006.0,9323.0,199705,1997,5,1990-1999
35,199705,2057340,5585343,1,WINIGAN,40.05,-92.88,4003.0,9253.0,199705,1997,5,1990-1999


In [11]:
# Re-order columns
lfulldata = lfulldata[['EVENT_ID', 'EPISODE_ID', 'LOCATION', 'LATITUDE', 'LONGITUDE', 'LATITUDE2', 'LONGITUDE2', 
                      'YEAR', 'MONTH', 'DECADE', 'YEARMONTH']]

lfulldata.head()

,EVENT_ID,EPISODE_ID,LOCATION,LATITUDE,LONGITUDE,LATITUDE2,LONGITUDE2,YEAR,MONTH,DECADE,YEARMONTH
12,5591463,2058233,DOUGLAS,34.17,-86.38,3410.0,8623.0,1997,1,1990-1999,199701
20,5585340,2057340,LAREDO,40.03,-93.45,4002.0,9327.0,1997,5,1990-1999,199705
21,5585341,2057340,HUMPHREYS,40.08,-93.35,4005.0,9321.0,1997,5,1990-1999,199705
34,5585342,2057340,PURDIN,40.10,-93.38,4006.0,9323.0,1997,5,1990-1999,199705
35,5585343,2057340,WINIGAN,40.05,-92.88,4003.0,9253.0,1997,5,1990-1999,199705


In [12]:
# Export the lfulldata as 1 CSV file
lfulldata.to_csv('../Resources/StormEventsLocationsALL.csv')